In [2]:
import pandas as pd


In [3]:
trips_csv ='~/Downloads/UKDA-5340-tab/tab/tripeul2017.tab'
households_csv = '~/Downloads/UKDA-5340-tab/tab/householdeul2017.tab'
individuals_csv = '~/Downloads/UKDA-5340-tab/tab/individualeul2017.tab'

In [4]:
travel_diaries_in = pd.read_csv(trips_csv, sep='\t')

/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (28,30,31,44,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
tds_2017 = travel_diaries_in.loc[travel_diaries_in.SurveyYear == 2017, :]
tds_2017.head()

,TripID,SurveyYear,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,JourSeq,HowComp_B01ID,...,TripTotalTime_ImpBin,TripTravTime_ImpBin,TripOrigUA2009_ImpBin,TripOrigUA1998_ImpBin,TripDestUA2009_ImpBin,TripDestUA1998_ImpBin,TripOrigCounty_ImpBin,TripDestCounty_ImpBin,TripOrigGOR_ImpBin,TripDestGOR_ImpBin
4238446,2017000001,2017,2017000001,2017000001,2017000001,2017000001,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
4238447,2017000002,2017,2017000002,2017000001,2017000001,2017000001,1,2,1,2,...,0,0,0,0,0,0,0,0,0,0
4238448,2017000003,2017,2017000002,2017000001,2017000001,2017000001,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0
4238449,2017000004,2017,2017000003,2017000001,2017000001,2017000001,1,3,1,2,...,0,0,0,0,0,0,0,0,0,0
4238450,2017000005,2017,2017000003,2017000001,2017000001,2017000001,1,3,2,2,...,0,0,0,0,0,0,0,0,0,0


In [14]:
tds_2017['uid'] = [str(x) + '_' + str(y) for x, y in zip(tds_2017['IndividualID'], tds_2017['DayID'])]

/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
tds_2017.head(20)

,TripID,SurveyYear,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,JourSeq,HowComp_B01ID,...,TripTravTime_ImpBin,TripOrigUA2009_ImpBin,TripOrigUA1998_ImpBin,TripDestUA2009_ImpBin,TripDestUA1998_ImpBin,TripOrigCounty_ImpBin,TripDestCounty_ImpBin,TripOrigGOR_ImpBin,TripDestGOR_ImpBin,uid
4238446,2017000001,2017,2017000001,2017000001,2017000001,2017000001,1,1,1,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000001
4238447,2017000002,2017,2017000002,2017000001,2017000001,2017000001,1,2,1,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000002
4238448,2017000003,2017,2017000002,2017000001,2017000001,2017000001,1,2,2,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000002
4238449,2017000004,2017,2017000003,2017000001,2017000001,2017000001,1,3,1,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000003
4238450,2017000005,2017,2017000003,2017000001,2017000001,2017000001,1,3,2,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000003
4238451,2017000006,2017,2017000004,2017000001,2017000001,2017000001,1,4,1,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000004
4238452,2017000007,2017,2017000004,2017000001,2017000001,2017000001,1,4,2,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000004
4238453,2017000008,2017,2017000005,2017000001,2017000001,2017000001,1,5,1,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000005
4238454,2017000009,2017,2017000005,2017000001,2017000001,2017000001,1,5,2,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000005
4238455,2017000010,2017,2017000006,2017000001,2017000001,2017000001,1,6,1,2,...,0,0,0,0,0,0,0,0,0,2017000001_2017000006


In [7]:
households_in = pd.read_csv(households_csv, sep='\t')
households_in.head()

/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23,35,38,40,41,42,43,52,57,58,159,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,HouseholdID,SurveyYear,PSUID,W0,W1,W2,W3,OutCom_B02ID,TWSDay,TWSMonth,...,JTimeGP_ImpBin,JTimeGroc_ImpBin,JTimeHosp_ImpBin,JTimePost_ImpBin,JTimePrimSch_ImpBin,JTimeSecSch_ImpBin,JTimeShopC_ImpBin,WalkBus_ImpBin,WalkRail_ImpBin,WalkRailAlt_ImpBin
0,2002000001,2002,2002000001,1,1,0.987670,0.968129,-10,23,1,...,,,,,,,,,,
1,2002000002,2002,2002000001,1,1,1.013688,1.136902,-10,17,1,...,,,,,,,,,,
2,2002000003,2002,2002000001,1,1,1.254002,1.167336,-10,15,1,...,,,,,,,,,,
3,2002000004,2002,2002000001,1,1,1.098194,1.101900,-10,30,1,...,,,,,,,,,,
4,2002000005,2002,2002000001,1,1,0.911825,0.899429,-10,21,1,...,,,,,,,,,,


In [8]:
individuals_in = pd.read_csv(individuals_csv, sep='\t')
individuals_in.head()

/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (42,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,225,226,227,228,229,230,231,232,233,234,235,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,629,696,697,698,699,700,701,702,703) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SurveyYear,IndividualID,HouseholdID,PSUID,VehicleID,W0,W1,W2,W3,PersNo,...,YNtCyB_DNA_B01ID,YNtCyB_DEAD_B01ID,EcoStat_ImpBin,IndIncome_ImpBin,IndWkCounty_ImpBin,IndWkGOR_ImpBin,IndWkUA1998_ImpBin,IndWkUA2009_ImpBin,Stat_ImpBin,WkPlace_ImpBin
0,2002,2002000001,2002000001,2002000001,2002000001,1,1,0.987670,0.968129,1,...,2,1,,,,,,,,
1,2002,2002000002,2002000001,2002000001,2002000002,1,1,0.987670,0.968129,2,...,2,1,,,,,,,,
2,2002,2002000003,2002000001,2002000001,,1,1,0.987670,0.968129,3,...,2,1,,,,,,,,
3,2002,2002000004,2002000001,2002000001,,1,1,0.987670,0.968129,4,...,2,1,,,,,,,,
4,2002,2002000005,2002000002,2002000001,,1,1,1.013688,1.136902,1,...,2,1,,,,,,,,


In [ ]:
travel_diaries_in

In [9]:
travel_diaries_in.columns

Index(['TripID', 'SurveyYear', 'DayID', 'IndividualID', 'HouseholdID', 'PSUID',
       'PersNo', 'TravDay', 'JourSeq', 'HowComp_B01ID', 'SeriesCall_B01ID',
       'ShortWalkTrip_B01ID', 'NumStages', 'NumStages_B01ID', 'MainMode_B03ID',
       'MainMode_B04ID', 'MainMode_B11ID', 'TripPurpose_B02ID',
       'TripPurpose_B04ID', 'TripDisIncSW', 'TripDisIncSW_B01ID',
       'TripDisExSW', 'TripDisExSW_B01ID', 'TripPurpFrom_B01ID',
       'TripPurpTo_B01ID', 'TripPurpose_B01ID', 'JJXSC', 'JOTXSC', 'JTTXSC',
       'JD', 'W5', 'W5xHh', 'TripStartHours', 'TripStartMinutes', 'TripStart',
       'TripStart_B01ID', 'TripStart_B02ID', 'TripEndHours', 'TripEndMinutes',
       'TripEnd', 'TripEnd_B01ID', 'TripEnd_B02ID', 'TripTotalTime',
       'TripTotalTime_B01ID', 'TripTravTime', 'TripTravTime_B01ID',
       'TripOrigGOR_B02ID', 'TripDestGOR_B02ID', 'TripOrigUA2009_B01ID',
       'TripDestUA2009_B01ID', 'ShortWalkTrip_ImpBin', 'TripPurpFrom_ImpBin',
       'TripPurpTo_ImpBin', 'TripPurpose_ImpBi


    pid - person ID
    hid - household ID
    seq - trip sequence number
    hzone - household zone
    ozone - trip origin zone
    dzone - trip destination zone
    purp - trip purpose
    mode - trip mode
    tst - trip start time (minutes)
    tet - trip end time (minutes)
    freq - weighting for representative population


In [18]:
travel_diaries = tds_2017[
    [
        'uid',
        'HouseholdID',
        'JourSeq',
        'TripPurpFrom_B01ID',
        'TripOrigUA2009_B01ID',
        'TripDestUA2009_B01ID',
        'TripPurpose_B01ID',
        'MainMode_B04ID',
        'TripStart',
        'TripEnd',
        'W5'
    ]
]

In [19]:
travel_diaries.head(20)

,uid,HouseholdID,JourSeq,TripPurpFrom_B01ID,TripOrigUA2009_B01ID,TripDestUA2009_B01ID,TripPurpose_B01ID,MainMode_B04ID,TripStart,TripEnd,W5
4238446,2017000001_2017000001,2017000001,1,23,530,530,1,1,1080,1100,1.0043
4238447,2017000001_2017000002,2017000001,1,1,530,530,1,1,15,35,1.02422
4238448,2017000001_2017000002,2017000001,2,23,530,530,1,1,1080,1100,1.02422
4238449,2017000001_2017000003,2017000001,1,1,530,530,1,1,15,35,1.03589
4238450,2017000001_2017000003,2017000001,2,23,530,530,1,1,1080,1100,1.03589
4238451,2017000001_2017000004,2017000001,1,1,530,530,1,1,15,35,1.03754
4238452,2017000001_2017000004,2017000001,2,23,530,530,1,1,1080,1100,1.03754
4238453,2017000001_2017000005,2017000001,1,1,530,530,1,1,15,35,1.03883
4238454,2017000001_2017000005,2017000001,2,23,530,530,1,1,1080,1100,1.03883
4238455,2017000001_2017000006,2017000001,1,1,530,530,1,1,15,35,1.05864


In [ ]:
travel_diaries.groupby('uid').

In [ ]:
travel_diaries.rename(columns={
    'HouseholdID': 'hid',
    'JourSeq': 'seq',
    'HHoldGOR_B02ID' : 'hzone',
    'TripPurpose_B02ID' : 'purp', 
    'MainMode_B03ID' : 'mode',
    'TripStart' : 'tst',
    'TripEnd' : 'tet'
})